# SOLID Design Principles

5 principles 

## Single Responsibility (Seperation of Concerns) SRP, SOC


a class/ type should only be responsible for one thing - e.g. a Journal class is responsible for storing items of a journal. if you then wanted to have save/ load functionality, that should be handled by a seperate object. This is so if you have a bunch of types that all need to be saved, you have one central location from which you can change the code, instead of editing all of them

The class should have a single reason to change, related to it's single responsibility


antipattern:

god object = class that does literally everything

In [ ]:
# our journal class should have one job - to store/remove the entries, and store the number of entries.
class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0

    def add_entry(self, text):
        self.entries.append(f"{self.count}: {text}")
        self.count += 1

    def remove_entry(self, pos):
        del self.entries[pos]

    def __str__(self):
        return "\n".join(self.entries)

    # break SRP - if we add persistance to the class, we are breaking the principle that it should do only one thing. 
    
    def save(self, filename):
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def load(self, filename):
        pass

    def load_from_web(self, uri):
        pass


In [ ]:
# instead we add a persistance manager that does the saving (and maybe loading). this can then be used for other classes, e.g. perhaps we have a Log class or whatever.

class PersistenceManager:
    def save_to_file(journal, filename):
        file = open(filename, "w")
        file.write(str(journal))
        file.close()


j = Journal()
j.add_entry("I cried today.")
j.add_entry("I ate a bug.")
print(f"Journal entries:\n{j}\n")

p = PersistenceManager()
#file = r'c:\temp\journal.txt'
#p.save_to_file(j, file)

# verify!
with open(file) as fh:
    print(fh.read())


## Open-Closed principle OCP

classes should be open for extension, closed for modification. so if you have a class and you want to add some functionality, you should not modify the existing class. 

utilise things like abstract base classes to define common functionality, and inherit these with new classes to extend your functionality, rather than adding specific methods to one class


In [ ]:
# base product classes and attrs

from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3


class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3


class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size


In [ ]:
# this is a bad example - we have to add a function every time we want a new filter, and the more criteria we have, the more functions we are going to have. will get crazy

class ProductFilter:
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p

    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p

    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.color == color and p.size == size:
                yield p

    # state space explosion
    # 3 criteria
    # c s w cs sw cw csw = 7 methods

    # OCP = open for extension, closed for modification



In [ ]:
# instead, we want to define two base classes to inherit from. these have methods to check for if a condition is satisfied, and a filter method that will take our specification and return the items that satisfy it

# this way we can define generic classes like AndSpecification, and use this to combine base specs like colour+size etc.

class Specification:
    def is_satisfied(self, item):
        pass

    # and operator makes life easier
    def __and__(self, other):
        return AndSpecification(self, other)


class Filter:
    def filter(self, items, spec):
        pass

In [ ]:
class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color

    def is_satisfied(self, item):
        return item.color == self.color


class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size

    def is_satisfied(self, item):
        return item.size == self.size

# this would be a bad way to do it, because what if we want to combine >2 specs?
    
# class AndSpecification(Specification):
#     def __init__(self, spec1, spec2):
#         self.spec2 = spec2
#         self.spec1 = spec1
#
#     def is_satisfied(self, item):
#         return self.spec1.is_satisfied(item) and \
#                self.spec2.is_satisfied(item)

class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args

    # here we use the map function to map each of the ags to the specification function. we wrap it in all() to return those that were true for all specifications
    def is_satisfied(self, item):
        return all(map(
            lambda spec: spec.is_satisfied(item), self.args))

# so we give our filter class a list of items, and a specification class that will return a true/false for those that are satisfied.
class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item

In [ ]:
# this is the example using the bad ones
apple = Product('Apple', Color.GREEN, Size.SMALL)
tree = Product('Tree', Color.GREEN, Size.LARGE)
house = Product('House', Color.BLUE, Size.LARGE)

products = [apple, tree, house]

pf = ProductFilter()
print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
    print(f' - {p.name} is green')


In [ ]:
#this is the example of the better one
bf = BetterFilter()

print('Green products (new):')
green = ColorSpecification(Color.GREEN)
for p in bf.filter(products, green):
    print(f' - {p.name} is green')

print('Large products:')
large = SizeSpecification(Size.LARGE)
for p in bf.filter(products, large):
    print(f' - {p.name} is large')

print('Large blue items:')
# large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))
large_blue = large & ColorSpecification(Color.BLUE)
for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue')

## Liskov substitution principle LSP

if you have some interface that takes some kind of base class, you should be able to stick a derived class in there and everything should still work

In [ ]:
# here we define a base class REctangle, with properties width and height

class Rectangle:
    def __init__(self, width, height):
        self._height = height
        self._width = width

    @property
    def area(self):
        return self._width * self._height

    def __str__(self):
        return f'Width: {self.width}, height: {self.height}'

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        self._height = value



In [ ]:
# we then subclass it with the square - since we know that the sides must be the same, then we force that if we change the height then the width must change. However this will fuck up our use_it function below, since that was already using the rectangle class.

class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)

    @Rectangle.width.setter
    def width(self, value):
        _width = _height = value

    @Rectangle.height.setter
    def height(self, value):
        _width = _height = value




In [ ]:
def use_it(rc):
    w = rc.width
    rc.height = 10  # unpleasant side effect
    expected = int(w * 10)
    print(f'Expected an area of {expected}, got {rc.area}')


rc = Rectangle(2, 3)
use_it(rc)

sq = Square(5)
use_it(sq)


in the above example if we wanted to fix this, we dont even really need a square class. we can just use rectangle and add a boolean that says true if square or not. or just yeet the setters in square

## interface segregation principle (ISP)

we don't want too many methods in one interface, it may be problematic as you will force subclasses to define methods that they do not need/ are not applicable.


In [ ]:
from abc import abstractmethod

# this is a bad example - what if we have an old printer that does not able to scan? we will still have to define a scan method- even if we raise an error they will still see a .scan() on the api and it can be messy// confusing


class Machine:
    def print(self, document):
        raise NotImplementedError()

    def fax(self, document):
        raise NotImplementedError()

    def scan(self, document):
        raise NotImplementedError()


# ok if you need a multifunction device
class MultiFunctionPrinter(Machine):
    def print(self, document):
        pass

    def fax(self, document):
        pass

    def scan(self, document):
        pass


class OldFashionedPrinter(Machine):
    def print(self, document):
        # ok - print stuff
        pass

    def fax(self, document):
        pass  # do-nothing

    def scan(self, document):
        """Not supported!"""
        raise NotImplementedError('Printer cannot scan!')


printer = OldFashionedPrinter()
printer.fax(123)  # nothing happens
printer.scan(123)  # oops!


In [ ]:
#instead it is better to have one abc per thing, e.g. printer, scanner, etc.
class Printer:
    @abstractmethod
    def print(self, document): pass


class Scanner:
    @abstractmethod
    def scan(self, document): pass
# same for Fax, etc.


In [ ]:

# we can then define subclasses that inherit whatever pieces they need
class MyPrinter(Printer):
    def print(self, document):
        print(document)

class Photocopier(Printer, Scanner):
    def print(self, document):
        print(document)

    def scan(self, document):
        pass  # something meaningful

#or we can even define new base classes that inherit the single base classes and re-overwrite the abstract methods. i am not sure why you would want to do this over just inheriting from the base classes though?
class MultiFunctionDevice(Printer, Scanner):  # , Fax, etc
    @abstractmethod
    def print(self, document):
        pass

    @abstractmethod
    def scan(self, document):
        pass


class MultiFunctionMachine(MultiFunctionDevice):
    def __init__(self, printer, scanner):
        self.printer = printer
        self.scanner = scanner

    def print(self, document):
        self.printer.print(document)

    def scan(self, document):
        self.scanner.scan(document)

## Dependency inversion principle (DIP)

high level modules should not depend on low level modules, they should depend on abstractions (ABCs)

In [ ]:
# this is what you should not do - the research is dependent on the relationships stored in the Relationships class- if we change this because we actually want to store them in a database, or store in a flat file or whatever, then our research class will be fucked as it is expecting the list/ tuple structure. 

In [1]:
from abc import abstractmethod
from enum import Enum


class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2


class Person:
    def __init__(self, name):
        self.name = name



class Relationships:  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name
                
class Research:
    # dependency on a low-level module directly
    # bad because strongly dependent on e.g. storage type

     #     # high-level: find all of john's children
    def __init__(self, relationships):
        relations = relationships.relations
        for r in relations:
            if r[0].name == 'John' and r[1] == Relationship.PARENT:
                print(f'John has a child called {r[2].name}.')

In [2]:

parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris.
John has a child called Matt.


instead we define a relationship browser base class, with method(s) we will want to use in our research. this way we define how the function works on the relationships class, and if the storage structure changes, we change the function there and any downstream dependencies will not know the difference.

what i don't get is how this does not break the single responsibility principle? Should we not have a seperate class for the relationships and the search methods?

In [ ]:
class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name): pass


class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name
                
                

In [ ]:
class Research:
    def __init__(self, browser):
        for p in browser.find_all_children_of("John"):
            print(f'John has a child called {p}')

parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

# Gamma Categorisation

design patterns split into 3 categories

 1) Creational Patterns
 > deal with the creation of objects
 
 > Explicit vs Implicit
 
 > Wholesale (one line) vs piecewise (step by step)
 
 2) Structural Patterns
 > concerned with the structure (e.g. class members)
 
 > many patterns are wrappers that mimic the underlying class
 
 > stress the importance of good api design
 
 3) Behavioral Patterns
 
 > they are all different, no central theme
 
 > typically unique in approach

# Builder

some objects are simple and can be initialised with one argument. initialising with 10 args is unweildy. instead opt for picewise constriction. 


builder is api that you can use to construct object instead to make it more succinct/ easy

## builder

imagine we want to create html code

In [ ]:

# if you want to build a simple HTML paragraph using a list
hello = 'hello'
parts = ['<p>', hello, '</p>']
print(''.join(parts))

# now I want an HTML list with 2 words in it
words = ['hello', 'world']
parts = ['<ul>']
for w in words:
    parts.append(f'  <li>{w}</li>')
parts.append('</ul>')
print('\n'.join(parts))

In [4]:
text = 'hello'
parts = ['<p>', text, '</p>']
print(''.join(parts))

<p>hello</p>


In [5]:
class HtmlElement:
    indent_size = 2

    def __init__(self, name="", text=""):
        self.name = name
        self.text = text
        self.elements = []

    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)

    def __str__(self):
        return self.__str(0)
    
    # we can add the create method here and expose the builder class directly
    @staticmethod
    def create(name):
        return HtmlBuilder(name)


class HtmlBuilder:
    #root is the thing we are building up
    __root = HtmlElement()

    def __init__(self, root_name):
        self.root_name = root_name
        self.__root.name = root_name

    # not fluent
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )

    # fluent - this is so we can chain together add_ commands
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        return self

    def clear(self):
        self.__root = HtmlElement(name=self.root_name)

    def __str__(self):
        return str(self.__root)



In [8]:


# ordinary non-fluent builder
builder = HtmlBuilder('ul')

#builder = HtmlElement.create('ul')
builder.add_child('li', 'hello')
builder.add_child('li', 'world')
print('Ordinary builder:')
print(builder)


Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [9]:

# fluent builder
builder.clear()
builder.add_child_fluent('li', 'hello').add_child_fluent('li', 'world')
print('Fluent builder:')
print(builder)


Fluent builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


## builder facets
what if a problem is so complicated that we need multiple builders?

in this example we will build a person

In [11]:
class Person:
    def __init__(self):
        print('Creating an instance of Person')
        # address
        self.street_address = None
        self.postcode = None
        self.city = None
        # employment info
        self.company_name = None
        self.position = None
        self.annual_income = None

    def __str__(self) -> str:
        return f'Address: {self.street_address}, {self.postcode}, {self.city}\n' +\
            f'Employed at {self.company_name} as a {self.postcode} earning {self.annual_income}'

# this is our base class
#if it is initialised with no person, then we want to start with a blank person
class PersonBuilder:  # facade
    def __init__(self, person=None):
        if person is None:
            self.person = Person()
        else:
            self.person = person

    @property
    def lives(self):
        return PersonAddressBuilder(self.person)

    @property
    def works(self):
        return PersonJobBuilder(self.person)

    def build(self):
        return self.person

# we want a sub builder for the job
# in this instance we don't want to create a new person, we want to super the base
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, company_name):
        self.person.company_name = company_name
        return self

    def as_a(self, position):
        self.person.position = position
        return self

    def earning(self, annual_income):
        self.person.annual_income = annual_income
        return self

# we want a sub builder for the address
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, street_address):
        self.person.street_address = street_address
        return self

    def with_postcode(self, postcode):
        self.person.postcode = postcode
        return self

    def in_city(self, city):
        self.person.city = city
        return self



In [26]:
# we create a blank person
pb = PersonBuilder()

# we build our person using our builders


p = pb\
    .lives\
        .at('123 London Road')\
        .in_city('London')\
        .with_postcode('SW12BC')\
    .works\
        .at('Fabrikam')\
        .as_a('Engineer')\
        .earning(123000)\
    .build()
print(p)



Creating an instance of Person
Address: 123 London Road, SW12BC, London
Employed at Fabrikam as a SW12BC earning 123000


In [27]:
# we access the address builder using the .lives attribute. since we are initing it with the existing self.person, it is not overwriting our person
print(pb.lives)

# since we are inheriting from the base class, we can do weird stuff like this- is this bad? should it be allowed to access these methods?
pp = pb.lives.lives.lives.lives.lives.lives.lives.at('Works still?').build()
print(pp)

Address: Works still?, SW12BC, London
Employed at Fabrikam as a SW12BC earning 123000


In [13]:
person2 = PersonBuilder().build()
print(person2)


Creating an instance of Person
Address: None, None, None
Employed at None as a None earning None


## builder inheritance

the above is bad since it breaks the open closed principle. if we add a new builder, we need to edit the base class to add it






In [ ]:
class Person:
    def __init__(self):
        self.name = None
        self.position = None
        self.date_of_birth = None

    def __str__(self):
        return f'{self.name} born on {self.date_of_birth} works as a {self.position}'

    @staticmethod
    def new():
        return PersonBuilder()

# this does nothing more than initialise a person, and return self.person with the build method
class PersonBuilder:
    def __init__(self):
        self.person = Person()

    def build(self):
        return self.person


    
class PersonInfoBuilder(PersonBuilder):
    def called(self, name):
        self.person.name = name
        return self

# we can now inherit from PersonInfoBuilder ad infinitum
class PersonJobBuilder(PersonInfoBuilder):
    def works_as_a(self, position):
        self.person.position = position
        return self


class PersonBirthDateBuilder(PersonJobBuilder):
    def born(self, date_of_birth):
        self.person.date_of_birth = date_of_birth
        return self




In [ ]:

# we can now init with the most derived class, and since we are cascading the inheritence, we can access the methods from the info and job builders

pb = PersonBirthDateBuilder()
me = pb\
    .called('Dmitri')\
    .works_as_a('quant')\
    .born('1/1/1980')\
    .build()  # this does NOT work in C#/C++/Java/...
print(me)

# this means that we never have to edit any of the old builders, we cans just inherit from another one.

## excersise

you are asked to implement the builder design pattrn for rendering simple chunks of code


e.g.

cb = CodeBuilder('Person').add_field('name', '""').add_field('age', '0')


this should create:

Class Person:
    def __init__(self):
        self.name = ''
        self.age = ''

In [55]:
class ClassElement():
    
    def __init__(self, name, value):
        self.name = name
        self.value = value
        
    def __str__(self):
        return('self.{} = {}'.format(name, value))
    

    
class Class():
    
    def __init__(self, root_name):
        self._root_name = root_name
        self._attrs = []
        self.indent = ' '

    
    def __str__(self):
        lines = []
        lines.append('class {}():'.format(self._root_name))
        
        if len(self._attrs) >0:
            line = '{}def __init__(self'.format(self.indent*4)
            attrs_lines = []
            for i in self._attrs:
                attrs_lines.append('{}self.{}= {}'.format(self.indent*8, i.name, i.value))
                line = line + ', {} = {}'.format(i.name, i.value)
            line = line + ('):')
            lines.append(line)
            lines.extend(attrs_lines)
        else:
            lines.append('{}pass'.format(self.indent*4))
        
        return('\n'.join(lines))
    
    
class CodeBuilder:
    
    def __init__(self, root_name):
        self.__class = Class(root_name)
        
        # todo

    def add_field(self,name, value):
        self.__class._attrs.append(ClassElement(name, value))
        return self

    def __str__(self):
        return self.__class.__str__()

In [56]:
cb = CodeBuilder('prick')

In [57]:
cb = cb.add_field('one', 1).add_field('two', 2).add_field('dgldsflkg', 'asfgtg')

In [58]:
print(cb)

class prick():
    def __init__(self, one = 1, two = 2, dgldsflkg = asfgtg):
        self.one= 1
        self.two= 2
        self.dgldsflkg= asfgtg


i also added the fields to the init of the class - maybe should not have tho

# Factories

why? 

- Sometimes object creation logic can be v long and convoluted. maybe we want to move the logic somewhere. 

- \_\_init\_\_ is always called init - cannot rename to give hint on what it will do.

- cannot overload it with same args with different names

- can turn init into optional parameter hell

- wholesale object creation (not piecewise like the builder)

- create seperate method (typically static method) this is factory method design pattern

- move to seperate class (factory. e.g we have class Foo, and FooFactory which creates different types of Foo objects)

- hierarchy of factories corresponding to hierarchy of types -> abstract factory


**Factory is a component responsible solely for the wholesale (not piecewise) creation of objects**







## Factory Method

In [59]:
#we want a class for a point in space
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
##    # what if we want to allow initialising from polar coords instead? We cannot redefine innit, it wil just override it. 

In [60]:
# BAD EXAMPLE 2

from enum import Enum
from math import *


class CoordinateSystem(Enum):
    CARTESIAN = 1
    POLAR = 2

class Point:
    # this is still not great, since it is ambiguous that a refers to x, b refers to y. Also if we want to add a new coordinate system, we will have to add a new one to our coordinateSystem class, and then modify this init. This breaks our open closed principle!
    def __init__(self, a, b, system=CoordinateSystem.CARTESIAN):
        if system == CoordinateSystem.CARTESIAN:
            self.x = a
            self.y = b
        elif system == CoordinateSystem.POLAR:
            self.x = a * sin(b)
            self.y = a * cos(b)



In [63]:
# here is better since we can rename the args in our static methods, 

class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "X: {}, Y: {}".format(self.x, self.y)
    @staticmethod
    def new_cartesian_point(x, y):
        return Point(x, y)

    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho * sin(theta), rho * cos(theta))




In [64]:
# we can use the default one
p= Point(2,3)


# we can access the static methods without instancing the class
p2 = Point.new_polar_point(1,2)

print(p, p2)

X: 2, Y: 3 X: 0.9092974268256817, Y: -0.4161468365471424


## Factory 

In [67]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f'x: {self.x}, y: {self.y}'

# take out factory methods to a separate class. this way we can give different names to the args - x and y become rho and theta for the polar

class PointFactory:
    @staticmethod
    def new_cartesian_point(x, y):
        return Point(x, y)

    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho * sin(theta), rho * cos(theta))


#PROBLEM - what if the Point initialiser changes? We will also have to change the point factory

# also how does client know that there is a factory that they should use? they will knot know unless you specify in documentation


In [68]:
p2 = PointFactory.new_cartesian_point(1, 2)
print(p2)

x: 1, y: 2


In [ ]:

# or you can expose factory through the type
p3 = Point.Factory.new_cartesian_point(5, 6)
p4 = Point.factory.new_cartesian_point(7, 8)
print(p1, p2, p3, p4)


In [69]:

# we can also put the factory as an inner class
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f'x: {self.x}, y: {self.y}'

    class PointFactory:
        #our methods don't really need to be static anymore either. this may be useful if we want to store some state in the factory, e.g we want to have some offset when calculating the polar coords
        def new_cartesian_point(self,x, y):
            return Point(x, y)

        def new_polar_point(sef,rho, theta):
            return Point(rho * sin(theta), rho * cos(theta))
        
    # we can also have an instance of the point factory class. This is more for convenience:
    factory = PointFactory()
    

In [70]:
p3 = Point.factory.new_cartesian_point(1,2)
print(p3)

x: 1, y: 2


factories can sometimes refer to any kind of thing that manufactures an object- e.g if a lambda func is creating obects it could be described as a factory. Mostly though it is referring to a class with factory methods

## Abstract Factory

we want to make an abstract base class for our factories. e.g. we have modellers deriving from a base modeller, we may also want modeller factories deriving from a base factory.

In [71]:
from abc import ABC
from enum import Enum, auto

# here is our abc for the drinks, and the derived classes from it 

class HotDrink(ABC):
    def consume(self):
        pass


class Tea(HotDrink):
    def consume(self):
        print('This tea is nice but I\'d prefer it with milk')


class Coffee(HotDrink):
    def consume(self):
        print('This coffee is delicious')




In [72]:
# here we have our abstract factory as an abc, which mandates the prepare method

class HotDrinkFactory(ABC):
    def prepare(self, amount):
        pass


class TeaFactory(HotDrinkFactory):
    # prepare is our factory method, returning instance of Tea
    def prepare(self, amount):
        print(f'Put in tea bag, boil water, pour {amount}ml, enjoy!')
        return Tea()


class CoffeeFactory(HotDrinkFactory):
    def prepare(self, amount):
        print(f'Grind some beans, boil water, pour {amount}ml, enjoy!')
        return Coffee()


In [75]:
def make_drink(type):
    if type == 'tea':
        return TeaFactory().prepare(200)
    
    elif type == 'coffee':
        return CoffeeFactory().prepare(400)
    else:
        return ValueError('need tea or coffee')

In [74]:
entry = input('what drink you like?')
drink = make_drink(entry)
drink.consume()

what drink you like?tea
Put in tea bag, boil water, pour 200ml, enjoy!
This tea is nice but I'd prefer it with milk


this is ok but it is not leveraging it being abstract -we are not really using the ABC other than mandating the prepare method 

instead we can organise it better:

In [76]:
class HotDrinkMachine:
    class AvailableDrink(Enum):  # violates OCP, what if we want a new kind of drink? we will have to modify this.
        COFFEE = auto()
        TEA = auto()

    factories = []
    initialized = False

    def __init__(self):
        if not self.initialized:
            self.initialized = True
            # we create a factory instance of each available drink in our list
            for d in self.AvailableDrink:
                name = d.name[0] + d.name[1:].lower()
                factory_name = name + 'Factory'
                factory_instance = eval(factory_name)()
                self.factories.append((name, factory_instance))





    def make_drink(self):
        print('Available drinks:')
        for f in self.factories:
            print(f[0])

        s = input(f'Please pick drink (0-{len(self.factories)-1}): ')
        
        idx = int(s)
        s = input(f'Specify amount: ')
        amount = int(s)
        return self.factories[idx][1].prepare(amount)




In [77]:

hdm = HotDrinkMachine()
drink = hdm.make_drink()
drink.consume()


Available drinks:
Coffee
Tea
Please pick drink (0-1): 0
Specify amount: 123
Grind some beans, boil water, pour 123ml, enjoy!
This coffee is delicious


In [78]:
drink

note that in the above example we actually dont really need the factory abc- if we removed it it would continue to work since our factories list in the hot drink machine does not care what type the objects are that we put in it. but it is good since it specifies the api you are expected to implement.

## exercise

You are given a class called Person. the person has 2 attributes- id and name.

please implement a PersonFactory, that has a not static create_person() method, that takes a person's name, and returns a person initialised with this name and an id.

the id of the person should be set as a 0 based index of the object created. so first person would have id 0, second = id 1, etc.


In [ ]:
# prompt

class Person:
    def __init__(self, id, name):
        self.id = id
        self.name = name

class PersonFactory:
    def create_person(self, name):
        # todo

In [89]:
# my answer

class Person:
    def __init__(self, id, name):
        self.id = id
        self.name = name
    def __str__(self):
        return "ID: {}, Name: {}".format(self.id, self.name)

        
class PersonFactory:
    initialized = False

    def __init__(self):
        if not self.initialized:
            self.initialized = True
            self._id = 0
       
    def create_person(self, name):
        person = Person(self._id, name)
        self._id +=1
        return person
    

In [90]:
factory = PersonFactory()
p1 = factory.create_person('jack')
print(p1)


p2 = factory.create_person('jill')
print(p2)


p3 = factory.create_person('santa claus')
print(p3)

ID: 0, Name: jack
ID: 1, Name: jill
ID: 2, Name: santa claus


In [ ]:
# answer from the class
from unittest import TestCase


class Person:
    def __init__(self, id, name):
        self.id = id
        self.name = name

class PersonFactory:
    id = 0

    def create_person(self, name):
        p = Person(PersonFactory.id, name)
        PersonFactory.id += 1
        return p

class Evaluate(TestCase):
    def test_exercise(self):
        pf = PersonFactory()

        p1 = pf.create_person('Chris')
        self.assertEqual(p1.name, 'Chris')
        self.assertEqual(p1.id, 0)

        p2 = pf.create_person('Sarah')
        self.assertEqual(p2.id, 1)


In [91]:
# why is it better to do what he did without an init? He still creates instance of it? I guess you just dont neeed it?

# Prototype

complicated objects are not designed from scratch - you dont go from 0 to a working car. Instead you are likely to have an existing design, that you can copy, customise it somehow, add any missing stuff etc.

in order to copy, we need 'deep copy' support

we want to make cloning convenient (e.g. with a factory)

prototype = partially or fully initialised object, that you can copy or clone, and make use of it



## prototype

In [1]:
import copy

# we have an address class for our person
class Address:
    def __init__(self, street_address, city, country):
        self.country = country
        self.city = city
        self.street_address = street_address

    def __str__(self):
        return f'{self.street_address}, {self.city}, {self.country}'



In [2]:
# we have a person that has a name and an address
class Person:
    def __init__(self, name, address):
        self.name = name
        self.address = address

    def __str__(self):
        return f'{self.name} lives at {self.address}'



In [3]:
john = Person("John", Address("123 London Road", "London", "UK"))
print(john)

John lives at 123 London Road, London, UK


In [5]:
# what if we want to make additional people?
jane = john
jane.name = 'Jane'
print(john, jane)

# this does not work- editing jane is also editing john. John and jane are referring to the same object

Jane lives at 123 London Road, London, UK Jane lives at 123 London Road, London, UK


In [6]:
john = Person("John", Address("123 London Road", "London", "UK"))
#instead we take a deepcopy to make a new object
# maybe we want to still keep using the Address object, but we want to sometimes customise it. if we did what we did before, we edit the original one
jane = copy.deepcopy(john)
jane.name = "Jane"
jane.address.street_address = "124 London Road"
print(john, jane)

#differmce with this and copy.copy:

# copy.copy is shallow copy- any object that is a reference will be copied over as a reference. So in the above example the Address part would be shared between john and jane still. Deep copy is creating new version of references.

John lives at 123 London Road, London, UK Jane lives at 124 London Road, London, UK


## prototype factory

sometimes we don't want to keep on copying prototypes- it is nice to have a seperate component that will do it for us.

In [7]:
import copy

# suppose we have employees in offices. they can be in a main office, or an auxilery office

class Address:
    def __init__(self, street_address, suite, city):
        self.suite = suite
        self.city = city
        self.street_address = street_address

    def __str__(self):
        return f'{self.street_address}, Suite #{self.suite}, {self.city}'

class Employee:
    def __init__(self, name, address):
        self.address = address
        self.name = name

    def __str__(self):
        return f'{self.name} works at {self.address}'


In [8]:


class EmployeeFactory:
    # we create prototypes of the main office and auxilery office employees
    
    main_office_employee = Employee("", Address("123 East Dr", 0, "London"))
    aux_office_employee = Employee("", Address("123B East Dr", 0, "London"))

    @staticmethod
    def __new_employee(proto, name, suite):
        #takes the prototype, and copies it. edits it with the info, and returns it
        result = copy.deepcopy(proto)
        result.name = name
        result.address.suite = suite
        return result

    @staticmethod
    def new_main_office_employee(name, suite):
        return EmployeeFactory.__new_employee(
            EmployeeFactory.main_office_employee,
            name, suite
        )

    @staticmethod
    def new_aux_office_employee(name, suite):
        return EmployeeFactory.__new_employee(
            EmployeeFactory.aux_office_employee,
            name, suite
        )


In [9]:

# this is how we would have done it before, if we wanted to make a new instance for John:
main_office_employee = Employee("", Address("123 East Dr", 0, "London"))
aux_office_employee = Employee("", Address("123B East Dr", 0, "London"))

john = copy.deepcopy(main_office_employee)
john.name = "John"
john.address.suite = 101
print(john)


# would prefer to write just one line of code
jane = EmployeeFactory.new_aux_office_employee("Jane", 200)
print(jane)



John works at 123 East Dr, Suite #101, London
Jane works at 123B East Dr, Suite #200, London


## Exercise

given the definitions shown in the below code, you are asked to implement Line.deepcopy() to perform a deep copy og the given Line object. This method should return a copy of Line that contains copies of it's start and end points

Note please do not confuse deep_copy() with \_\_deepcopy\_\_() !

In [15]:
# prompt
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

class Line:
    def __init__(self, start=Point(), end=Point()):
        self.start = start
        self.end = end

    def deep_copy(self):
        # TODO
        pass

In [28]:

#my answer
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

class Line:
    def __init__(self, start=Point(), end=Point()):
        self.start = start
        self.end = end

    def deep_copy(self):
        cp = copy.deepcopy(self)
        return cp
    
    def __str__(self):
        return('{} {} {} {}'.format(self.start.x, self.start.y, self.end.x, self.end.y))

In [29]:
p1 = Point(1,1)
p2 = Point(2,2)

l1 = Line(p1,p2)

In [33]:
l2 = l1.deep_copy()
l2.start = Point(3,3)
l2.end = Point(3,3)

In [34]:
print(l1)
print(l2)

1 1 2 2
3 3 3 3


In [36]:
# his answer
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

class Line:
    def __init__(self, start=Point(), end=Point()):
        self.start = start
        self.end = end

    def deep_copy(self):
        new_start = Point(self.start.x, self.start.y)
        new_end = Point(self.end.x, self.end.y)
        return Line(new_start, new_end)

In [35]:
# i dunno if mine is ok?? it works so i dont see why not